Explore the vault dataset. The Vault has 3 sub-dataset: Class, Method and Inline.

### THE VAULT FUNCTION

In [3]:
from datasets import load_dataset

dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["train/small"], languages=["java"], streaming=True)
dataset

IterableDatasetDict({
    train_small: IterableDataset({
        features: ['hexsha', 'repo', 'path', 'license', 'language', 'identifier', 'return_type', 'original_string', 'original_docstring', 'docstring', 'docstring_tokens', 'code', 'code_tokens', 'short_docstring', 'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
        num_shards: 2
    })
})

In [8]:
i = 0
for sample in iter(dataset['train_small']): 
    code = sample['code']
    docstring = sample['docstring']
    project_name = sample['repo']
    method_name = sample['identifier']
    print(f"Method '{method_name}' of project '{project_name}'")
    print(f"comment:\n{docstring}")
    print(code)
    i += 1
    if i >= 100:
        break

Method 'addFavorite' of project 'watchful-sky/WatchfulSky'
comment:
Method to adda satellite to the favorites list.
Writes the name and TLE data of the selected satellite to a specific file
Filename is dependant on the type of satellite.
It was necessary to implement in this way because of how the expandable list view is populated.

@param name     Name of Satellite
@param line1    TLE line 1
@param line2    TLE line 2
@param fileName File containing satellite
public static void addFavorite(String name, String line1, String line2, String fileName, Context context) {
        try {
            boolean favExists = Favorites.contains(name, fileName, context);
            if (!favExists) {
                FileOutputStream fostream = context.openFileOutput("favorites_" + fileName, Context.MODE_APPEND);
                OutputStreamWriter oswriter = new OutputStreamWriter(fostream);
                BufferedWriter bwriter = new BufferedWriter(oswriter);
                bwriter.write(name);
    

### THE VAULT INLINE

In [1]:
from datasets import load_dataset

dataset = load_dataset("Fsoft-AIC/the-vault-inline", languages=['java'], streaming=True)
dataset

IterableDatasetDict({
    train: IterableDataset({
        features: ['hexsha', 'repo', 'path', 'license', 'language', 'identifier', 'code', 'code_tokens', 'original_comment', 'comment', 'comment_tokens', 'start_point', 'end_point', 'prev_context', 'next_context'],
        num_shards: 75
    })
})

In [11]:
i = 0
prev_method_name = None
for sample in dataset['train']:
    code = sample['code']
    comment = sample['comment']
    method_name = sample['identifier']
    if method_name != prev_method_name:
        print("\n")
        print(code)
    prev_context = sample['prev_context']
    next_context = sample['next_context']
    print(f"{comment}")
    prev_method_name = method_name
    i += 1
    if i >= 100:
        break



@ProcessElement
    public void processElement(@Element Long rowkey, OutputReceiver<Mutation> out) {
      long timestamp = System.currentTimeMillis();

      // Pad and reverse the rowkey for more distributed writes
      String numberFormat = "%0" + 30 + "d";
      String paddedRowkey = String.format(numberFormat, rowkey);
      String reversedRowkey = new StringBuilder(paddedRowkey).reverse().toString();
      Put row = new Put(Bytes.toBytes(reversedRowkey));

      // Generate random bytes
      long rowSize = MB_PER_ROW * ONE_MB;
      byte[] randomData = new byte[(int) rowSize];

      random.nextBytes(randomData);
      row.addColumn(
          Bytes.toBytes(COLUMN_FAMILY),
          Bytes.toBytes("C"),
          timestamp,
          randomData);
      out.output(row);
    }
Pad and reverse the rowkey for more distributed writes
Generate random bytes


void testWebhookUrl(final CloseableHttpClient client, final String webhookUrl, final ResultBuilder builder) {
        final Ht

### THE VAULT CLASS

In [4]:
from datasets import load_dataset

dataset = load_dataset("Fsoft-AIC/the-vault-class", languages=['java'], streaming=True)
dataset

IterableDatasetDict({
    train: IterableDataset({
        features: ['hexsha', 'repo', 'path', 'license', 'language', 'identifier', 'original_docstring', 'docstring', 'docstring_tokens', 'code', 'code_tokens', 'short_docstring', 'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
        num_shards: 60
    })
})

In [5]:
i = 0
for sample in dataset['train']:
    code = sample['code']
    docstring = sample['docstring']
    project_name = sample['repo']
    method_name = sample['identifier']
    print(f"Class '{method_name}' of project '{project_name}'")
    print(f"comment:\n{docstring}")
    print(code)
    i += 1
    if i >= 100:
        break

Class 'GnssStatus' of project 'ghillert/gnss-nmea-demo'
comment:
Basic status information of the GNSS (GPS, Galileo etc.) data.
public class GnssStatus {

	private volatile Double altitude;
	private volatile Double longitude;
	private volatile Double latitude;

	private volatile GpsFixQuality fixQuality;
	private volatile Map<GnssProvider, Integer> satelliteCount = new ConcurrentHashMap<>();
	private volatile GpsFixStatus gpsFixStatus;

	private volatile Double calculatedHorizontalAccuracyInMeters;
	private volatile Double ubloxHorizontalAccuracyInMeters;
	private volatile Double ubloxVerticalAccuracyInMeters;

	public Double getAltitude() {
		return this.altitude;
	}
	public void setAltitude(Double altitude) {
		this.altitude = altitude;
	}
	public GpsFixQuality getFixQuality() {
		return this.fixQuality;
	}
	public void setFixQuality(GpsFixQuality fixQuality) {
		this.fixQuality = fixQuality;
	}
	public Map<GnssProvider, Integer> getSatelliteCount() {
		return this.satelliteCount;
	}